<a href="https://colab.research.google.com/github/AmandinChyba/Project1-Fields-2022/blob/main/Walk_on_Boundary_and_BIEM/WalkOnBoundary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
from jax import lax
from jax import random

import jax.numpy as jnp
import numpy as np

import time

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

from functools import partial

In [2]:
def unit_circle_boundary(x, y):
  return jnp.sqrt(jnp.power(x, 2) + jnp.power(y, 2)) - 1

In [272]:
def getMiddle(p0, p1):
  return (p1[0]-p0[0])/2 + p0[0], (p1[1]-p0[1])/2 + p0[1]

def getDistance(p0, p1):
  # distance squared
  return jnp.power(p0[0]-p1[0],2) + jnp.power(p0[0]-p1[0],2)

def binaryRootSearch(x, y, angle, distanceF, epochs, ray):
  #error = - 2 * jnp.power(0.5, epochs)

  theta = angle
  #if not ray:
  #  if distanceF(x-0.01*jnp.cos(theta), y-0.01*jnp.sin(theta)) > 0:
  #      theta -= jnp.pi

  lower = (x,y)
  #upper = (x+2.1*jnp.cos(theta),y+2.1*jnp.sin(theta)) # pick a point of distance 2 on the line
  if ray:
    upper = (x+2.1*jnp.cos(theta),y+2.1*jnp.sin(theta))
  else:
    upper = (x-2.1*jnp.cos(theta),y-2.1*jnp.sin(theta))
  
  for i in range(epochs):
    middle = getMiddle(lower, upper)
    if distanceF(middle[0], middle[1]) > 0:
      upper = middle
    else:
      lower = middle
  
  #return getMiddle(lower, upper)
  return upper

In [270]:
def wob(g, distanceF, x0, y0, t, epochs, key):
  angles = random.uniform(key, shape=(t, 1)) * 2 * jnp.pi

  runningEst = 0
  runningSign = 1
  #x, y = initial_point(x0, y0, angles[0], distanceF, 0.001)
  x, y = binaryRootSearch(x0, y0, angles[0], distanceF, epochs, True)
  #print('(',x[0],',',y[0],')')

  for angle in angles[1:]:
    runningEst += runningSign * g(x, y)
    runningSign *= -1
    
    #x,y = binaryRootSearch(x, y, angle, distanceF, epochs, False)
    p0 = binaryRootSearch(x, y, angle, distanceF, epochs, False)
    p1 = binaryRootSearch(x, y, angle, distanceF, epochs, True)
    # pick the farther x,y pair as the new location
    if getDistance((x,y),p0) > getDistance((x,y),p1):
      x,y = p0
    else:
      x,y = p1

    #x, y = get_point(x, y, angle, distanceF, 0.001)
    #print('(',x[0],',',y[0],')')

  runningEst *= 2
  runningEst += runningSign * g(x,y)

  return runningEst

In [ ]:
#%%time
g = lambda x, y : y
distanceF = unit_circle_boundary
x0 = 0
y0 = -0.9
batches = 1000
t = 10
epochs = 50
key = random.PRNGKey(2)
keys = random.split(key, batches)

ans = np.empty(batches)
for i in range(batches):
  value = wob(g, distanceF, x0, y0, t, epochs, keys[i])
  ans[i] = jnp.array(value)
  if (i % 50 == 0 and i != 0):
    print('soln:', jnp.mean(ans[:i]))

mean = jnp.mean(ans)
print(mean)